In [ ]:
!pip install transformers
!pip install conllu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.3/8.3 MB 83.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.7/311.7 kB 46.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 39.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 738.9/738.9 kB 83.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 70.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 169.0/169.0 kB 43.6 MB/s eta 0:00:00

[notice] A new release of pip is available: 23.0.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip


In [ ]:
import torch
import torch.nn as nn
import numpy as np
from collections import Counter

from transformers import BertTokenizer, BertModel

from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import accuracy_score
import math
import linecache
import json
import os

/shared-libs/python3.9/py/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
json_subdir_list = os.listdir('/datasets/dylansdrive/Amazon_JSON_Repos/')
print(json_subdir_list)
review_collection = np.array([])
line_counts = [497577, 231392, 3410019, 2375, 34278, 77071, 6898955, 1143860, 2972, 6739590, 169781, 11285464, 1128437, 1443755, 27164983, 1711519, 494485, 2277, 3176, 5269]
for i in json_subdir_list:
  with open (f'/datasets/dylansdrive/Amazon_JSON_Repos/{i}/{i}', 'r') as review_extract:
      print(i)
      # Temp  TODO: Split Books_5 so it can be loaded w/o memory issues
      if (i != 'Books_5.json'):
          k = 0
          # Temp  TODO: Sample more
          while k < 50:
              review_collection = np.append(review_collection, json.loads(review_extract.readline()))
              k += 1
print(review_collection)

['Arts_Crafts_and_Sewing_5.json', 'Appliances_5.json', 'Industrial_and_Scientific_5.json', 'Luxury_Beauty_5.json', 'Clothing_Shoes_and_Jewelry_5.json', 'Video_Games_5.json', 'Electronics_5.json', 'Magazine_Subscriptions_5.json', 'Movies_and_TV_5.json', 'Grocery_and_Gourmet_Food_5.json', 'Automotive_5.json', 'Musical_Instruments_5.json', 'Cell_Phones_and_Accessories_5.json', 'All_Beauty_5.json', 'AMAZON_FASHION_5.json', 'Books_5.json', 'CDs_and_Vinyl_5.json', 'Gift_Cards_5.json', 'Home_and_Kitchen_5.json', 'Digital_Music_5.json']
Arts_Crafts_and_Sewing_5.json
Appliances_5.json
Industrial_and_Scientific_5.json
Luxury_Beauty_5.json
Clothing_Shoes_and_Jewelry_5.json
Video_Games_5.json
Electronics_5.json
Magazine_Subscriptions_5.json
Movies_and_TV_5.json
Grocery_and_Gourmet_Food_5.json
Automotive_5.json
Musical_Instruments_5.json
Cell_Phones_and_Accessories_5.json
All_Beauty_5.json
AMAZON_FASHION_5.json
Books_5.json
CDs_and_Vinyl_5.json
Gift_Cards_5.json
Home_and_Kitchen_5.json
Digital_Musi

In [ ]:
tokenizer = BertTokenizer.from_pretrained("bert-base-cased")

tokenizer_config.json: 100%|██████████| 29.0/29.0 [00:00<00:00, 8.44kB/s]
vocab.txt: 100%|██████████| 213k/213k [00:00<00:00, 74.4MB/s]
tokenizer.json: 100%|██████████| 436k/436k [00:00<00:00, 36.1MB/s]
config.json: 100%|██████████| 570/570 [00:00<00:00, 609kB/s]


In [ ]:
np.random.shuffle(review_collection)
review_train, review_test = review_collection[0:int(len(review_collection) * 0.75)], review_collection[int(len(review_collection) * 0.75):]


In [ ]:
def tokenize_reviews(data, tokenizer):
  tokenized = []
  for inst in data:
    n_inst = {
      'review': tokenizer(inst['reviewText'], return_tensors='pt'),
      'orig_review': inst['reviewText'],
    }
    n_inst['str_tokenized_review'] = tokenizer.convert_ids_to_tokens(n_inst['review']['input_ids'][0])
    tokenized.append(n_inst)

  return tokenized

In [ ]:
tokenized_review_data = tokenize_reviews(review_train, tokenizer)

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=0b63f49d-657e-40c3-abbf-efbd6029b8d0' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>